In [1]:
%load_ext autoreload
%autoreload 2

from loss.cross_entropy_2d import CrossEntropy2d
import torch
import numpy as np

from torch.nn import NLLLoss
import torch.nn.functional as F


def cross_entropy2d(input, target, weight=None, size_average=True):
    n, c, h, w = input.size()
    nt, ht, wt = target.size()

    # Handle inconsistent size between input and target
    if h != ht and w != wt:  # upsample labels
        input = F.interpolate(input, size=(ht, wt), mode="bilinear", align_corners=True)

    input = input.transpose(1, 2).transpose(2, 3).contiguous().view(-1, c)
    target = target.view(-1)
    loss = F.cross_entropy(
        input, target, weight=weight, size_average=size_average, ignore_index=250
    )
    return loss
  
device = torch.device("cpu")
loss_fn = CrossEntropy2d()

lbl = torch.tensor([[1,2], [2,1]]).to(device)
print(lbl)

idx_1 = torch.tensor([0.0, 0.6, 0.3])
idx_2 = torch.tensor([0.0, 5.0, 1.0])

out = torch.zeros(3,2,2)
out[:,0,0] = idx_1
out[:,0,1] = idx_2
out[:,1,0] = idx_2
out[:,1,1] = idx_1
out = out.unsqueeze(0)
print(out)
print("....")
l_o = loss_fn(out, lbl.unsqueeze(0))

p_v, pred = out.max(1)
print(pred.squeeze())
print(p_v.squeeze())

l_2 = cross_entropy2d(out, lbl.unsqueeze(0))
print(l_o)
print(l_2)


out_r = out.reshape((1,3,4))
print(out_r.shape)
#l_3 = NLLLoss(, lbl.unsqueeze(0).reshape((1,4)))

tensor([[1, 2],
        [2, 1]])
tensor([[[[0.0000, 0.0000],
          [0.0000, 0.0000]],

         [[0.6000, 5.0000],
          [5.0000, 0.6000]],

         [[0.3000, 1.0000],
          [1.0000, 0.3000]]]])
....
tensor([[1, 1],
        [1, 1]])
tensor([[0.6000, 5.0000],
        [5.0000, 0.6000]])
tensor(2.4266)
tensor(2.4266)
torch.Size([1, 3, 4])


/usr/local/google/home/ammarh/.local/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [2]:
%load_ext autoreload
%autoreload 2

from loss.cross_entropy_2d import CrossEntropy2d
import torch
import torch.nn as nn
import numpy as np

from torch.nn import NLLLoss

# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()
# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
print(f"ffffff {data.requires_grad}")
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)
# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
optimizer = torch.optim.Adam(conv.parameters())

print(target[1,:,:])

print("Init")
output = m(conv(data))
_, pred = output.max(1)
print(pred[1,:,:])


for i in range(1000):
  optimizer.zero_grad()
  output = loss(m(conv(data)), target)
  print(output.item())
  output.backward()
  optimizer.step()

print("After")
output = m(conv(data))
_, pred = output.max(1)
print(pred[1,:,:])



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ffffff False
tensor([[3, 0, 0, 3, 1, 3, 3, 3],
        [3, 1, 3, 1, 0, 2, 3, 1],
        [3, 1, 3, 3, 3, 1, 0, 2],
        [1, 1, 0, 2, 2, 3, 0, 3],
        [3, 3, 0, 3, 1, 1, 0, 1],
        [0, 2, 1, 0, 3, 0, 2, 3],
        [1, 2, 0, 1, 3, 1, 2, 0],
        [1, 1, 0, 0, 2, 3, 3, 3]])
Init
tensor([[3, 3, 3, 2, 3, 3, 0, 3],
        [2, 0, 3, 3, 0, 0, 3, 0],
        [0, 0, 2, 3, 0, 0, 0, 1],
        [0, 2, 3, 3, 2, 0, 3, 1],
        [2, 2, 3, 1, 3, 3, 3, 3],
        [1, 3, 2, 2, 3, 0, 1, 3],
        [1, 2, 2, 1, 0, 0, 1, 3],
        [1, 2, 2, 1, 1, 2, 1, 2]])
1.4820446968078613
1.4702770709991455
1.4586683511734009
1.4472209215164185
1.4359337091445923
1.4248080253601074
1.4138447046279907
1.4030402898788452
1.3923956155776978
1.3819080591201782
1.3715779781341553
1.3614013195037842
1.3513784408569336
1.3415071964263916
1.3317875862121582
1.3222181797027588
1.3127954006195068
1.30351984500885
1.294389

In [3]:
import numpy as np

k = np.array([[1,2],[3,4]], dtype=np.uint8)
print(k)

k[k >2 ] = -1
print(k)

[[1 2]
 [3 4]]
[[  1   2]
 [255 255]]
